# Justicia : formal fairness verifier
This goal of this tutorial is to illustrate how to formally verify different fairness metrics of a prediction algorithm in presence of compound sensitive groups. In this tutorial, the inputs are 
- a prediction algorithm represented as a CNF (Conjunctive Normal Form) formula,
- the probability distribution of input features (computed from finite sampled dataset).

Fairness literature has developed multiple definitions of fairness, of which we verify (1) disparate impact and (2) statistical parity definitions. For an elaborate discussion, we refer to [[1]](https://arxiv.org/pdf/2009.06516.pdf).


In the following, we show three different prediction algorithms (classifiers) and verify fairness on each of them.

## A. CNF classifier

At first, we consider a CNF classifier that can learn a CNF formula in a dataset with binary class labels. In this tutorial, we use the CNF learner from [[2]](https://bishwamittra.github.io/publication/imli-ghosh.pdf). 

In [1]:
# standard library
import sklearn.metrics
from sklearn.model_selection import train_test_split
from pyrulelearn.imli import imli
import sys

# From this framework
import justicia.utils
from justicia.metrics import Metric

# data
sys.path.append("..")
from data.objects.adult import Adult
from data.objects.titanic import Titanic

### Prepare a dataset

In [2]:
verbose = True
dataset = Adult(verbose=verbose, config=5)
df = dataset.get_df()
# discretize
df =  justicia.utils.get_discretized_df(df, columns_to_discretize=dataset.continuous_attributes, verbose=False)
# get X,y
X = df.drop(['target'], axis=1)
y = df['target']
# one-hot
X = justicia.utils.get_one_hot_encoded_df(X,X.columns.to_list(), verbose=False)
X

Sensitive attributes: ['race', 'age']
-number of samples: (before dropping nan rows) 32561
-number of samples: (after dropping nan rows) 32561


,sex,age_0,age_1,age_2,age_3,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,...,capital-gain_1,capital-gain_3,capital-loss_0,capital-loss_1,capital-loss_2,capital-loss_3,hours-per-week_0,hours-per-week_1,hours-per-week_2,hours-per-week_3
0,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
1,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
4,1,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,1,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
32557,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
32558,1,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
32559,0,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0


### Train a model

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=2) # 70% training and 30% test

clf = imli(num_clause=2, data_fidelity=10, work_dir="../data/", rule_type="CNF", verbose=False)
clf.fit(X_train.values, y_train.values)

In [4]:
print("learned rule:")
print(clf.get_rule(X_train.columns.to_list()))
print("\nTrain Accuracy:", sklearn.metrics.accuracy_score(clf.predict(X_train.values),y_train.values))
print("Test Accuracy:", sklearn.metrics.accuracy_score(clf.predict(X_test.values),y_test.values))

learned rule:
capital-gain_1 OR capital-gain_3 OR capital-loss_1 OR capital-loss_2 AND
education-num_3 OR hours-per-week_2 OR not capital-gain_0

Train Accuracy: 0.7757302757302758
Test Accuracy: 0.7688056493705864


### Formally measure different fairness metrics

Justicia reduces the verification problem to solving appropriately designed SSAT (Stochastic Satisfiability) instances. Two different encodings have been proposed in \[1\]

- enumeration (Enum) based encoding and
- learning (Learn) based encoding.

In the Enum encoding, we formally measure fairness metrics such as disparate impact and statistical parity on each combination of compound sensitive groups. Consider sensitive attributes = (Race, Sex) where Race $\in$ {Black, White} and Sex $\in$ {Male, Female}. In Enum encoding, we compute metrics for all four combinations: Black-Male, Black-Female, White-Black, and White-Female. 

In the Learn encoding, the (SSAT) solver finds the most favored (least discriminated) and least favored (most discriminated) group efficeintly without us enumerating explicitly.


First we show Enum encoding.




In [5]:
metric_enum = Metric(model=clf, data=X_test, sensitive_attributes=dataset.known_sensitive_attributes, verbose=False, encoding="Enum")
metric_enum.compute()
# details
# print(metric_enum)
# print()
print("Disparate Impact:", metric_enum.disparate_impact_ratio)
print("Statistical Parity:", metric_enum.statistical_parity_difference)
print("Time taken", metric_enum.time_taken, "seconds")

Disparate Impact: 1
Statistical Parity: 0.0
Time taken 1.8231730461120605 seconds


Now we show Learn encoding.

In [6]:
metric_learn = Metric(model=clf, data=X_test, sensitive_attributes=dataset.known_sensitive_attributes, verbose=False, encoding="Learn")
metric_learn.compute()
print("Disparate Impact:", metric_learn.disparate_impact_ratio)
print("Statistical Parity:", metric_learn.statistical_parity_difference)
print("Most favored group:", metric_learn.most_favored_group)
print("Least favored group:", metric_learn.least_favored_group)
print("Time taken", metric_learn.time_taken, "seconds")

Disparate Impact: 1.000000926244106
Statistical Parity: -1.9999999961717307e-08
Most favored group: {'race_Amer-Indian-Eskimo': ('==', 1), 'age_0': ('==', 1)}
Least favored group: {'race_Amer-Indian-Eskimo': ('==', 1), 'age_0': ('==', 1)}
Time taken 0.20220017433166504 seconds


We see that both encodings have same output, the difference is in the efficiency of computing these metrics. Experimentally, the Learn encoding is superior than the Enum encoding.

### Considering the correlation between sensitive and non-sensitive attributes

In the previous two encodings (Learn and Enum), we consider independence of all attributes, which is unlikely in practical ML problems. Therefore, we propose Enum-correlation encoding that considers the conditional dependency of non-sensitive attributes on each compound group.  


In [7]:
metric_enum = Metric(model=clf, data=X_test, sensitive_attributes=['sex'], verbose=False, encoding="Enum").compute()
print("Without correlation->")
print("Disparate Impact:", metric_enum.disparate_impact_ratio)
print("Statistical Parity:", metric_enum.statistical_parity_difference)
print("Most favored group:", metric_enum.most_favored_group)
print("Least favored group:", metric_enum.least_favored_group)

print("\n\n")
print("With correlation->")
metric_enum_cor = Metric(model=clf, data=X_test, sensitive_attributes=['sex'], verbose=False, encoding="Enum-correlation").compute()
print("Disparate Impact:", metric_enum_cor.disparate_impact_ratio)
print("Statistical Parity:", metric_enum_cor.statistical_parity_difference)
print("Most favored group:", metric_enum_cor.most_favored_group)
print("Least favored group:", metric_enum_cor.least_favored_group)


print("\n\n")
print("Recalling the classification rule in CNF")
rule = clf.get_rule(X_train.columns.to_list())
print(rule)
print("\nSensitive attribute in classifier?", "sex" in rule)

Without correlation->
Disparate Impact: 1
Statistical Parity: 0.0
Most favored group: {'sex': ('==', 1)}
Least favored group: {'sex': ('==', 1)}



With correlation->
Disparate Impact: 0.36338477803088115
Statistical Parity: 0.017751990000000002
Most favored group: {'sex': ('!=', 1)}
Least favored group: {'sex': ('==', 1)}



Recalling the classification rule in CNF
capital-gain_1 OR capital-gain_3 OR capital-loss_1 OR capital-loss_2 AND
education-num_3 OR hours-per-week_2 OR not capital-gain_0

Sensitive attribute in classifier? False


In this example, we see that without considering correlation among attributes, no discremination is detected. For example, the most favored group and the least favored group are identical (i.e., disparate impact = 1 and statistical parity = 0). However, when we consider correlation, there is a clear discremination. 

### References:

\[1\] Ghosh B, Basu D, Meel KS. Justicia: A Stochastic SAT Approach to Formally Verify Fairness.

\[2\] Ghosh B, Meel KS. IMLI: An incremental framework for MaxSAT-based learning of interpretable classification rules.